In [22]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator
from typing import List

from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from cv_builder.models import Resume
from dotenv import load_dotenv

load_dotenv()


llm = AzureChatOpenAI(
    deployment_name="gpt-35-16k",
    temperature=1,
    # verbose=False,
    # streaming=True,

)

# Set up a parser + inject instructions into the prompt template.
pydantic_parser = PydanticOutputParser(pydantic_object=Resume)

format_instructions = pydantic_parser.get_format_instructions()

template_string = """
You are a professional resume builder who specialized in building resumes for postgraduate applications, including master's and PhD's. 
I will provide you several piecies of information about my background and experience. You will then extract my personal information, education, work experience, research experience, project experience, and personal statement.
Below is the piece of information you will extract from:
```{texts}```

Then extract my personal information, education, work experience, research experience, project experience, and personal statement as much as you can.
{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template_string)



In [30]:
texts = """
Name: John Doe | Email: john.doe@example.com | Phone: 1234567890
""" 

In [31]:
messages = prompt.format_messages(texts=texts, 
                                format_instructions=format_instructions)

In [32]:
messages

[HumanMessage(content='\nYou are a professional resume builder who specialized in building resumes for postgraduate applications, including master\'s and PhD\'s. \nI will provide you several piecies of information about my background and experience. You will then extract my personal information, education, work experience, research experience, project experience, and personal statement.\nBelow is the piece of information you will extract from:\n```\nName: John Doe | Email: john.doe@example.com | Phone: 1234567890\n```\n\nThen extract my personal information, education, work experience, research experience, project experience, and personal statement as much as you can.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}\nthe object {"foo": ["bar", "baz"]} is a well-formatt

In [33]:
output = llm(messages)

In [27]:
output.content

'{\n  "personal_info": {\n    "name": "John Doe",\n    "email": "john.doe@example.com",\n    "phone": "1234567890"\n  },\n  "education": [\n    {\n      "degree": "Undergraduate",\n      "university": "China Three Gorges University",\n      "graduation_date": "2022"\n    }\n  ],\n  "work_experience": [\n    {\n      "company": "College of Mechanical and Power Engineering\'s Youth League Committee",\n      "position": "New Media Operations Department",\n      "start_date": "Unknown",\n      "end_date": "Unknown",\n      "summary": "Managed the committee\'s official social media account",\n      "bullet_points": []\n    }\n  ],\n  "research_experience": [\n    {\n      "organization": "Unknown",\n      "position": "Undergraduate researcher",\n      "start_date": "Unknown",\n      "end_date": "Unknown",\n      "summary": "Conducted research and completed an undergraduate thesis on the topic \'Design of Water Turbine Guide Vane Mechanism\'",\n      "bullet_points": []\n    }\n  ],\n  "proj

In [28]:
resume = pydantic_parser.parse(output.content)

resume

Resume(personal_info=PersonalInfo(name='John Doe', email='john.doe@example.com', phone='1234567890'), education=[Education(degree='Undergraduate', university='China Three Gorges University', graduation_date='2022')], work_experience=[WorkExperience(company="College of Mechanical and Power Engineering's Youth League Committee", position='New Media Operations Department', start_date='Unknown', end_date='Unknown', summary="Managed the committee's official social media account", bullet_points=[])], research_experience=[ResearchExperience(organization='Unknown', position='Undergraduate researcher', start_date='Unknown', end_date='Unknown', summary="Conducted research and completed an undergraduate thesis on the topic 'Design of Water Turbine Guide Vane Mechanism'", bullet_points=[])], project_experience=[ProjectExperience(name='Thermal Power Plant Design', start_date='Unknown', end_date='Unknown', summary='Collaborated with a team to design a thermal power plant for a course assignment', bu

In [29]:
import json
with open("input.json", "w") as f:
    json.dump(resume.dict(), f)